In [9]:
import pandas as pd
import numpy as np 

In [10]:
df_2019 = pd.read_csv('../data/race_data分割/race_data_2019.csv',index_col=0)
df_2020_2021 = pd.read_csv('../data/race_data分割/race_data_2020_2021.csv',index_col=0)
df_2012_2013 = pd.read_csv('../data/race_data分割/race_data_2012_2013.csv',index_col=0)
df_2014_2018 = pd.read_csv('../data/race_data分割/race_data_2014_2018.csv',index_col=0)
#githubにアップする際にはscrape用の方を修正したので不足データはない様にしていますが、フォルダ内にあるデータは一部不足データがあるためこの様な形で結合しています。
df_husoku = pd.read_csv('../data/race_data分割/race_data_husoku.csv',index_col=0)
df_race  = pd.concat([df_2012_2013,df_2014_2018,df_2019,df_2020_2021,df_husoku])


In [11]:
#カラムの半角スペースを削除　
df_race.rename(columns=lambda s: s.replace( ' ',''),inplace=True)
#可視化用にタイムfloat型でを秒にしておく(タイムはDashboard表示の際に見栄えが良くなるため取っておく)
df_race['time']=df_race['タイム'].str[0].astype(float)*60 + df_race['タイム'].str[2].astype(float)*10 + df_race['タイム'].str[3:].astype(float)
#馬体重をレース前体重と増減とをわける(馬体重は元々の方は消す)
df_race['体重']=df_race['馬体重'].str[:3]
df_race['増減']=df_race['馬体重'].str[3:]
#増減に残った()を消す
df_race['増減']=df_race['増減'].str.replace('(','').str.replace(')','')
df_race.drop(columns='馬体重',inplace=True)
#性齢列を性別列、年齢列に分割後、性齢列を削除
df_race['性別']=df_race['性齢'].str[0]
df_race['年齢']=df_race['性齢'].str[1:]
df_race.drop(columns='性齢',inplace=True)

In [12]:
df_race

,着順,枠番,馬番,馬名,斤量,騎手,タイム,着差,単勝,人気,調教師,性別,年齢,horse_id,jockey_id,trainer_id,time,体重,増減
201201010101,1,7,8,アウトシャイン,54.0,池添謙一,1:30.0,NaN,1.7,1.0,[西] 安田隆行,牝,2,2010101358,1032,438,90.0,442,0
201201010101,2,8,10,マイネルブルズアイ,54.0,津村明秀,1:30.4,2.1/2,38.7,8.0,[東] 武藤善則,牡,2,2010101786,1092,1064,90.4,452,-4
201201010101,3,8,9,マイネルゴスホーク,54.0,三浦皇成,1:30.5,1/2,7.6,4.0,[東] 古賀慎明,牡,2,2010102157,1122,1079,90.5,478,-8
201201010101,4,7,7,ユキカゼ,54.0,荻野琢真,1:30.7,1.1/2,23.9,7.0,[西] 牧田和弥,牡,2,2010103377,1112,1123,90.7,494,+12
201201010101,5,2,2,スピーディシャープ,54.0,小林徹弥,1:30.8,クビ,136.7,10.0,[西] 目野哲也,牡,2,2010104671,722,387,90.8,414,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202109021212,12,8,15,オーバーディリバー,55.0,横山武史,1:25.2,1,5.5,3.0,[東] 萩原清,牡,5,2016100182,1170,1017,85.2,482,0
202109021212,13,7,13,ハルサカエ,52.0,坂井瑠星,1:25.3,1/2,175.7,16.0,[東] 大竹正博,牝,5,2016103079,1163,1102,85.3,512,+2
202109021212,14,5,9,アプルーヴァル,54.0,和田竜二,1:25.6,1.3/4,21.6,10.0,[西] 今野貞一,牡,6,2015104950,1018,1128,85.6,512,0
202109021212,15,2,3,カバジェーロ,54.0,国分恭介,1:25.8,1,74.4,12.0,[西] 橋口慎介,牡,4,2017101830,1124,1154,85.8,494,0


In [13]:
#このままだと計測不良が邪魔をしてint型に変換できないため、全データをnumericにし、計測不能はNoneに置き換え、その後0埋めをしてint型にする
#Noneがあってもint型にできない。
df_race['増減']=pd.to_numeric(df_race['増減'],errors='coerce').fillna(0).astype(int)

In [14]:
df_race['体重'] = pd.to_numeric(df_race['体重'],errors='coerce').fillna(0).astype(int)

In [15]:
df_race['着順'] = pd.to_numeric(df_race['着順'],errors='coerce').fillna(0).astype(int)

In [16]:
df_race.info()

<class 'pandas.core.frame.DataFrame'>
Index: 488126 entries, 201201010101 to 202109021212
Data columns (total 19 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   着順          488126 non-null  int64  
 1   枠番          488126 non-null  int64  
 2   馬番          488126 non-null  int64  
 3   馬名          488126 non-null  object 
 4   斤量          488126 non-null  float64
 5   騎手          488126 non-null  object 
 6   タイム         484111 non-null  object 
 7   着差          449816 non-null  object 
 8   単勝          488126 non-null  object 
 9   人気          486441 non-null  float64
 10  調教師         488126 non-null  object 
 11  性別          488126 non-null  object 
 12  年齢          488126 non-null  int64  
 13  horse_id    488126 non-null  int64  
 14  jockey_id   488126 non-null  int64  
 15  trainer_id  488126 non-null  int64  
 16  time        484111 non-null  float64
 17  体重          488126 non-null  int64  
 18  増減          488126 non-null  int

In [18]:
df_race.to_csv('race_data_2012_2021.csv',header=True)